In [1]:
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Cloning https://github.com/huggingface/transformers.git to c:\users\alvin\appdata\local\temp\pip-req-build-yf96p45a
  Resolved https://github.com/huggingface/transformers.git to commit 22b41b3f8a5cdb37e686d18d8d9a24eb98a331ec
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\alvin\AppData\Local\Temp\pip-req-build-yf96p45a'


In [2]:
import librosa

audio_file_path = 'PrelimNoTranscription.mp3'
audio_array, sampling_rate = librosa.load(audio_file_path, sr=16000)

In [3]:
audio_data = {
    "array": audio_array,
    "sampling_rate": sampling_rate
}

audio_data, len(audio_data['array'])


({'array': array([ 1.2109997e-05,  2.7982878e-05,  2.4542742e-05, ...,
         -6.6963089e-06, -4.1230073e-06,  1.9476836e-06], dtype=float32),
  'sampling_rate': 16000},
 20367035)

In [4]:
import torch
from datasets import load_dataset

# #### "openai/whisper-small"
# from transformers import WhisperProcessor, WhisperForConditionalGeneration
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# model.config.forced_decoder_ids = None

#### pipeline method
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device)

model_id = "openai/whisper-small"
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)


cuda:0


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    return_timestamps=False,
    torch_dtype=torch_dtype,
    device=device,
)

    # max_new_tokens=128,
    # chunk_length_s=30,
    # batch_size=16,

In [6]:
# ## whisper-small
# input_features = processor(audio_data["array"], sampling_rate=audio_data["sampling_rate"], return_tensors="pt").input_features
# # generate token ids
# predicted_ids = model.generate(input_features)
# # decode token ids to text
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
# transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
# transcription


## pipeline method
transcription = pipe(audio_data)
transcription

c:\Users\alvin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\whisper\generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Users\alvin\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:697: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
The attention mask is not set and cannot be inferre

{'text': " obvious is what is a cryptocurrency? So this word was kind of invented 10 years ago when, I don't know how many of you know the origin story of where Bitcoin came from, but basically a pseudonym on the internet dropped a paper and some open source code in a forum on an email list and said, hey, I have this idea for this thing called Bitcoin. It's kind of like electronic cash. Here's how I think it could work. And here is some code if you want to run it and become part of this peer-to-peer network. We don't know who this person is. This person is basically virtually disappeared from the internet and from the world. But it's created something that has captured so many people's imaginations and has sort of, depending on how you measure it, created billions and billions of dollars of economic value and sort of inspired a lot of people to think about how to use this technology to solve a myriad of different problems, not just electronic payments. So cryptocurrencies and the techn

In [7]:
import json

exDict = {'exDict': transcription}

with open('PrelimTranscript.txt', 'w') as file:
     file.write(json.dumps(exDict))


In [10]:
from datasets import load_dataset

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]
result = pipe(sample)
print(result["text"])


Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, simile is drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Layton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Up Guards and Atom paintings, and Mason's exquisite idles are as national as a jingle poem. Mr. Birkitt Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back, before he says, like a shampoo We're in a Turkish bath. Next man!


In [11]:
dataset[0]["audio"]

{'path': '0d38672e0bbdbdc460af55b8bb84a15b2730db2819f2af64f9c777d4d586f2de',
 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00024414, 0.00048828,
        0.0005188 ]),
 'sampling_rate': 16000}

In [15]:
result = pipe(sample, return_timestamps=True)
print(result["chunks"])

ValueError: When passing a dictionary to AutomaticSpeechRecognitionPipeline, the dict needs to contain a "raw" key containing the numpy array representing the audio and a "sampling_rate" key, containing the sampling_rate associated with that array